In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cardiovascular_Disease"
cohort = "GSE256539"

# Input paths
in_trait_dir = "../../input/GEO/Cardiovascular_Disease"
in_cohort_dir = "../../input/GEO/Cardiovascular_Disease/GSE256539"

# Output paths
out_data_file = "../../output/preprocess/Cardiovascular_Disease/GSE256539.csv"
out_gene_data_file = "../../output/preprocess/Cardiovascular_Disease/gene_data/GSE256539.csv"
out_clinical_data_file = "../../output/preprocess/Cardiovascular_Disease/clinical_data/GSE256539.csv"
json_path = "../../output/preprocess/Cardiovascular_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Digital Spatial Profiling identifies distinct molecular signatures of vascular lesions in Pulmonary Arterial Hypertension"
!Series_summary	"Idiopathic Pulmonary Arterial Hypertension (IPAH) is a severe human disease, characterized by extensive pulmonary vascular remodeling due to plexiform and obliterative lesions, media hypertrophy, and alterations of adventitia. The objective of the study was to test the hypothesis that microscopic IPAH vascular lesions express unique molecular profiles, which collectively are different from control pulmonary arteries. We used digital spatial transcriptomics to profile the genome-wide differential transcriptomic signature of key pathological lesions (plexiform, obliterative, intima+media hypertrophy, and adventitia) in IPAH lungs (n= 11)  and compared these data to the intima+media and adventitia of control pulmonary artery (n=5). The IPAH lesions and pulmonary artery compartments were defined by the analyses of

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Availability assessment based on background information

# This is a gene expression profiling study using digital spatial transcriptomics
# The study mentions "genome-wide differential transcriptomic signature"
# This clearly indicates gene expression data is available
is_gene_available = True

# 2. Clinical feature extraction and conversion functions

# From sample characteristics, we need to analyze what data is available
# The dictionary shows 'individual' IDs and 'batch' information, but not disease status directly
# From the background information, we know this is a case-control study of IPAH vs controls

# 2.1 Determine row numbers for clinical features
# Looking at the sample characteristics, we can infer trait information from individual IDs
trait_row = 0  # We'll infer disease status from individual IDs

# Age and gender information are not provided in the sample characteristics
age_row = None  # Age data not available
gender_row = None  # Gender data not available

# 2.2 Define conversion functions for clinical features

def convert_trait(value):
    """
    Convert individual IDs to binary trait values.
    Based on the background information, we can infer:
    - IPAH patients have IDs with patterns like AH*, VA*, UC*, etc.
    - Control samples have IDs with patterns like CC*
    
    Returns:
    - 1 for IPAH cases
    - 0 for controls
    - None for unclear cases
    """
    if ':' in value:
        # Extract the value after the colon and strip whitespace
        value = value.split(':', 1)[1].strip()
    
    # Based on the background, control subjects would be in the control pulmonary artery group
    # The IDs "CC*" appear to be controls based on context and naming convention
    if value.startswith('CC'):
        return 0  # Control
    else:
        return 1  # IPAH case

def convert_age(value):
    """
    Convert age values to continuous numerical values.
    Not implemented as age data is not available.
    """
    return None

def convert_gender(value):
    """
    Convert gender values to binary (0 for female, 1 for male).
    Not implemented as gender data is not available.
    """
    return None

# 3. Save initial metadata about dataset usability
is_trait_available = trait_row is not None

validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if available
if trait_row is not None:
    # This assumes clinical_data is provided by previous steps
    try:
        # Load clinical data (this should have been done in previous steps)
        # Let's ensure it's available before proceeding
        if 'clinical_data' not in locals():
            # Try to load from the input directory
            clinical_file = os.path.join(in_cohort_dir, "sample_characteristics.csv")
            if os.path.exists(clinical_file):
                clinical_data = pd.read_csv(clinical_file)
            else:
                # Create from the sample characteristics dictionary if needed
                sample_chars_dict = {0: ['individuial: AH006', 'individuial: AH015', 'individuial: UA013', 'individuial: BA023', 'individuial: BA009', 'individuial: AH014', 'individuial: VA008', 'individuial: UC010', 'individuial: UC007', 'individuial: AH001', 'individuial: VA010', 'individuial: UA005', 'individuial: BA035', 'individuial: CC016', 'individuial: BA021', 'individuial: UM003'], 
                                    1: ['batch: plate3', 'batch: plate2', 'batch: plate1', 'batch: hWTA_20210810T192_DSP-1012340068802']}
                clinical_data = pd.DataFrame(sample_chars_dict)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview and save clinical features
        print("Clinical Features Preview:")
        print(preview_df(clinical_features))
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save clinical features
        clinical_features.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical features saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical features: {e}")
else:
    print("Clinical data not available. Skipping clinical feature extraction.")


Clinical Features Preview:
{'GSM8105899': [1.0], 'GSM8105900': [1.0], 'GSM8105901': [1.0], 'GSM8105902': [1.0], 'GSM8105903': [1.0], 'GSM8105904': [1.0], 'GSM8105905': [1.0], 'GSM8105906': [1.0], 'GSM8105907': [1.0], 'GSM8105908': [1.0], 'GSM8105909': [1.0], 'GSM8105910': [1.0], 'GSM8105911': [1.0], 'GSM8105912': [1.0], 'GSM8105913': [1.0], 'GSM8105914': [1.0], 'GSM8105915': [1.0], 'GSM8105916': [1.0], 'GSM8105917': [1.0], 'GSM8105918': [1.0], 'GSM8105919': [1.0], 'GSM8105920': [1.0], 'GSM8105921': [1.0], 'GSM8105922': [1.0], 'GSM8105923': [1.0], 'GSM8105924': [1.0], 'GSM8105925': [1.0], 'GSM8105926': [1.0], 'GSM8105927': [1.0], 'GSM8105928': [1.0], 'GSM8105929': [1.0], 'GSM8105930': [1.0], 'GSM8105931': [1.0], 'GSM8105932': [1.0], 'GSM8105933': [1.0], 'GSM8105934': [1.0], 'GSM8105935': [1.0], 'GSM8105936': [1.0], 'GSM8105937': [1.0], 'GSM8105938': [1.0], 'GSM8105939': [1.0], 'GSM8105940': [1.0], 'GSM8105941': [1.0], 'GSM8105942': [1.0], 'GSM8105943': [1.0], 'GSM8105944': [1.0], 'GSM81

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Cardiovascular_Disease/GSE256539/GSE256539_series_matrix.txt.gz


Gene data shape: (8273, 220)
First 20 gene/probe identifiers:
Index(['A2M', 'A4GALT', 'AAAS', 'AACS', 'AAGAB', 'AAK1', 'AAMDC', 'AAMP',
       'AARS1', 'AARSD1', 'AATF', 'ABCA10', 'ABCA13', 'ABCA3', 'ABCA6',
       'ABCA8', 'ABCB10', 'ABCB9', 'ABCC1', 'ABCC3'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers in the output, I can see they are already human gene symbols (like A2M, AAAS, ABCC1, etc.)
# These are standard HUGO gene symbols, not probe IDs or other identifiers that would need mapping

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols
print(f"Original gene data shape: {gene_data.shape}")

try:
    # Attempt to normalize gene symbols
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"Normalized gene data shape: {gene_data_normalized.shape}")
except Exception as e:
    print(f"Gene normalization failed: {e}")
    # If normalization fails, use the original gene data
    gene_data_normalized = gene_data.copy()
    print(f"Using original gene data with shape: {gene_data_normalized.shape}")

# Save the gene expression data 
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# 2. Recreate clinical features from scratch
def convert_trait(value):
    """
    Convert individual IDs to binary trait values.
    Based on the background information:
    - IPAH patients have IDs with patterns like AH*, VA*, UC*, etc.
    - Control samples have IDs with patterns like CC*
    """
    if ':' in value:
        # Extract the value after the colon and strip whitespace
        value = value.split(':', 1)[1].strip()
    
    # Based on the background, control subjects would be in the control pulmonary artery group
    # The IDs "CC*" appear to be controls based on context and naming convention
    if value.startswith('CC'):
        return 0  # Control
    else:
        return 1  # IPAH case

# Reload clinical data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
_, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

try:
    # Create clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data, 
        trait=trait, 
        trait_row=0,  # Using individual ID information as identified in Step 2
        convert_trait=convert_trait,
        age_row=None,  # No age information available
        convert_age=None,
        gender_row=None,  # No gender information available
        convert_gender=None
    )
    
    print(f"Clinical features shape: {clinical_features.shape}")
    print("Clinical features preview:")
    print(preview_df(clinical_features))
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    
    # Check if we have valid clinical data
    if clinical_features.isna().all().all():
        print("Clinical features contain only NaN values. Cannot proceed with linking.")
        raise ValueError("No valid clinical data found")
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data_normalized)
    print(f"Linked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty DataFrame")
    
    # 4. Handle missing values
    linked_data_clean = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")
    
    # Check if we still have data after handling missing values
    if linked_data_clean.shape[0] == 0 or linked_data_clean.shape[1] <= 1:
        raise ValueError("Dataset is empty after handling missing values")
    
    # 5. Check for bias in the dataset
    is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)
    
    # 6. Conduct final quality validation
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data_clean,
        note="Dataset contains gene expression data from IPAH (Idiopathic Pulmonary Arterial Hypertension) patients and controls."
    )
    
    # 7. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data_clean.to_csv(out_data_file, index=True)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset deemed not usable for associative studies. Linked data not saved.")

except Exception as e:
    print(f"Error in data processing: {e}")
    # Ensure we still validate and record the dataset as not usable
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,
        df=pd.DataFrame(),
        note=f"Processing error: {str(e)}"
    )
    print("Dataset deemed not usable due to processing errors. Linked data not saved.")

Original gene data shape: (8273, 220)
Normalized gene data shape: (8271, 220)


Gene expression data saved to ../../output/preprocess/Cardiovascular_Disease/gene_data/GSE256539.csv
Clinical features shape: (1, 220)
Clinical features preview:
{'GSM8105899': [1.0], 'GSM8105900': [1.0], 'GSM8105901': [1.0], 'GSM8105902': [1.0], 'GSM8105903': [1.0], 'GSM8105904': [1.0], 'GSM8105905': [1.0], 'GSM8105906': [1.0], 'GSM8105907': [1.0], 'GSM8105908': [1.0], 'GSM8105909': [1.0], 'GSM8105910': [1.0], 'GSM8105911': [1.0], 'GSM8105912': [1.0], 'GSM8105913': [1.0], 'GSM8105914': [1.0], 'GSM8105915': [1.0], 'GSM8105916': [1.0], 'GSM8105917': [1.0], 'GSM8105918': [1.0], 'GSM8105919': [1.0], 'GSM8105920': [1.0], 'GSM8105921': [1.0], 'GSM8105922': [1.0], 'GSM8105923': [1.0], 'GSM8105924': [1.0], 'GSM8105925': [1.0], 'GSM8105926': [1.0], 'GSM8105927': [1.0], 'GSM8105928': [1.0], 'GSM8105929': [1.0], 'GSM8105930': [1.0], 'GSM8105931': [1.0], 'GSM8105932': [1.0], 'GSM8105933': [1.0], 'GSM8105934': [1.0], 'GSM8105935': [1.0], 'GSM8105936': [1.0], 'GSM8105937': [1.0], 'GSM8105938': [1.0

Linked data shape after handling missing values: (220, 8272)
For the feature 'Cardiovascular_Disease', the least common label is '0.0' with 12 occurrences. This represents 5.45% of the dataset.
The distribution of the feature 'Cardiovascular_Disease' in this dataset is fine.



Linked data saved to ../../output/preprocess/Cardiovascular_Disease/GSE256539.csv
